In [6]:


# --- GRU & LSTM Models ---
class GRUModel(nn.Module):
    def __init__(self, input_size=2, hidden_size=64, num_layers=1, output_len=5, dropout=0.1):
        super().__init__()
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_len * 2)
    def forward(self, x):
        _, hn = self.rnn(x)
        out = self.fc(hn[-1])
        return out.view(-1, 5, 2)

class LSTMModel(nn.Module):
    def __init__(self, input_size=2, hidden_size=64, num_layers=1, output_len=5, dropout=0.1):
        super().__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_len * 2)
    def forward(self, x):
        _, (hn, _) = self.rnn(x)
        out = self.fc(hn[-1])
        return out.view(-1, 5, 2)

# --- Loss and Metrics ---
mse_loss = nn.MSELoss()

def velocity_loss(pred, target):
    return nn.functional.mse_loss(pred[:, 1:] - pred[:, :-1], target[:, 1:] - target[:, :-1])

def compute_metrics(pred, target):
    ade = torch.mean(torch.norm(pred - target, dim=2))
    fde = torch.mean(torch.norm(pred[:, -1] - target[:, -1], dim=1))
    mae = torch.mean(torch.abs(pred - target))
    return ade.item(), fde.item(), mae.item()

# --- Training Function ---
def train_model(model, name, epochs=100):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    best_val_loss = float('inf')
    patience = 5
    epochs_without_improvement = 0
    checkpoint_path = f"best_{name}.pt"

    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs} - [{name}]")
        model.train()
        total_loss = 0
        for xb, yb in tqdm(train_dl, leave=False):
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            pred = model(xb)
            loss = mse_loss(pred, yb) + 0.1 * velocity_loss(pred, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item() * xb.size(0)

        model.eval()
        val_loss, ade, fde, mae = 0, 0, 0, 0
        with torch.no_grad():
            for xb, yb in val_dl:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss = mse_loss(pred, yb) + 0.1 * velocity_loss(pred, yb)
                val_loss += loss.item() * xb.size(0)
                batch_ade, batch_fde, batch_mae = compute_metrics(pred, yb)
                ade += batch_ade * xb.size(0)
                fde += batch_fde * xb.size(0)
                mae += batch_mae * xb.size(0)

        train_loss = total_loss / len(train_dl.dataset)
        val_loss = val_loss / len(val_dl.dataset)
        ade /= len(val_dl.dataset)
        fde /= len(val_dl.dataset)
        mae /= len(val_dl.dataset)
        scheduler.step(val_loss)

        print(f"[{name}] Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f} | "
              f"ADE: {ade:.4f}, FDE: {fde:.4f}, MAE: {mae:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
            torch.save(model.state_dict(), checkpoint_path)
            print("✔️  Saved best model to", checkpoint_path)
        else:
            epochs_without_improvement += 1
            print(f"⚠️  No improvement ({epochs_without_improvement}/5)")

        if epochs_without_improvement >= patience:
            print("⏹️ Early stopping triggered.")
            break

    return best_val_loss, ade, fde, mae

# --- Define Configs Automatically ---
configs = []
for rnn_type in ['GRU', 'LSTM']:
    for hidden in [32, 64, 128]:
        for layers in [1, 2]:
            dropout = 0.1 if layers == 1 else 0.3
            name = f"{rnn_type}_{hidden}h_{layers}l"
            model_class = GRUModel if rnn_type == 'GRU' else LSTMModel
            model = model_class(hidden_size=hidden, num_layers=layers, dropout=dropout).to(device)
            configs.append((name, model))

# --- Run All Models ---
results = {}
for name, model in configs:
    print(f"\n🔧 Training model: {name}")
    val_loss, ade, fde, mae = train_model(model, name)
    results[name] = {'val_loss': val_loss, 'ADE': ade, 'FDE': fde, 'MAE': mae}

# --- Print Summary ---
print("\n📊 Final Results Summary:")
for name, metrics in sorted(results.items(), key=lambda x: x[1]['val_loss']):
    print(f"{name}: Val Loss={metrics['val_loss']:.4f}, ADE={metrics['ADE']:.4f}, "
          f"FDE={metrics['FDE']:.4f}, MAE={metrics['MAE']:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Prepared dataset: X=(609284, 10, 2), Y=(609284, 5, 2)
Using device: cuda

🔧 Training model: GRU_32h_1l

Epoch 1/100 - [GRU_32h_1l]


[GRU_32h_1l] Train Loss: 0.4831, Val Loss: 0.0134 | ADE: 0.0778, FDE: 0.1081, MAE: 0.0452
✔️  Saved best model to best_GRU_32h_1l.pt

Epoch 2/100 - [GRU_32h_1l]


[GRU_32h_1l] Train Loss: 0.0117, Val Loss: 0.0119 | ADE: 0.0732, FDE: 0.1021, MAE: 0.0454
✔️  Saved best model to best_GRU_32h_1l.pt

Epoch 3/100 - [GRU_32h_1l]


[GRU_32h_1l] Train Loss: 0.0108, Val Loss: 0.0135 | ADE: 0.0937, FDE: 0.1329, MAE: 0.0572
⚠️  No improvement (1/5)

Epoch 4/100 - [GRU_32h_1l]


...

[LSTM_128h_2l] Train Loss: 0.0058, Val Loss: 0.0067 | ADE: 0.0439, FDE: 0.0654, MAE: 0.0257
⚠️  No improvement (1/5)

Epoch 29/100 - [LSTM_128h_2l]


[LSTM_128h_2l] Train Loss: 0.0058, Val Loss: 0.0069 | ADE: 0.0473, FDE: 0.0687, MAE: 0.0283
⚠️  No improvement (2/5)

Epoch 30/100 - [LSTM_128h_2l]


[LSTM_128h_2l] Train Loss: 0.0058, Val Loss: 0.0067 | ADE: 0.0448, FDE: 0.0657, MAE: 0.0261
⚠️  No improvement (3/5)

Epoch 31/100 - [LSTM_128h_2l]


[LSTM_128h_2l] Train Loss: 0.0056, Val Loss: 0.0068 | ADE: 0.0448, FDE: 0.0667, MAE: 0.0264
⚠️  No improvement (4/5)

Epoch 32/100 - [LSTM_128h_2l]


[LSTM_128h_2l] Train Loss: 0.0055, Val Loss: 0.0068 | ADE: 0.0444, FDE: 0.0648, MAE: 0.0256
⚠️  No improvement (5/5)
⏹️ Early stopping triggered.

📊 Final Results Summary:
LSTM_128h_2l: Val Loss=0.0067, ADE=0.0444, FDE=0.0648, MAE=0.0256
LSTM_64h_2l: Val Loss=0.0069, ADE=0.0449, FDE=0.0662, MAE=0.0260
LSTM_64h_1l: Val Loss=0.0074, ADE=0.0416, FDE=0.0646, MAE=0.0235
GRU_128h_1l: Val Loss=0.0075, ADE=0.0424, FDE=0.0654, MAE=0.0241
GRU_64h_1l: Val Loss=0.0076, ADE=0.0436, FDE=0.0676, MAE=0.0250
GRU_128h_2l: Val Loss=0.0077, ADE=0.0660, FDE=0.0826, MAE=0.0375
GRU_32h_1l: Val Loss=0.0077, ADE=0.0433, FDE=0.0673, MAE=0.0248
LSTM_128h_1l: Val Loss=0.0078, ADE=0.0457, FDE=0.0711, MAE=0.0257
LSTM_32h_1l: Val Loss=0.0085, ADE=0.0581, FDE=0.0864, MAE=0.0330
GRU_64h_2l: Val Loss=0.0112, ADE=0.1448, FDE=0.1519, MAE=0.0961
LSTM_32h_2l: Val Loss=0.0122, ADE=0.3500, FDE=0.3730, MAE=0.2217
GRU_32h_2l: Val Loss=0.0128, ADE=0.2003, FDE=0.2295, MAE=0.1343


In [2]:
import torch 
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"